# Анализ пространственных данных. Домашнее задание №2

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/My Drive/made/geo')

Mounted at /content/drive


Мягкий дедлайн: __4 ноября 2020 г. 23:59__

Жесткий дедлайн (со штрафом в _50%_ от количества набранных вами за ДЗ баллов): __5 ноября 2020 г. 08:59__

Визуализация "чего-либо" __без__ выполненного основного задания оценивается в __0 баллов__

ФИО: `Акимов Алексей Андреевич`

Группа: `DS-12`

## Задание №1. Горячая точка (алгоритм - 10 баллов, визуализация - 10 баллов).

Генерируйте рандомные точки на планете Земля до тех пор, пока не попадете на территорию ``Афганистана``

1. Вы можете использовать функции принадлжености точки полигону и расстояния от точки до полигона (в метрах)
2. Предложите не наивный алгоритм поиска (генерировать __напрямую__ точку из полигона границ Афганистана __запрещено__)

In [7]:
!pip install geopandas

In [3]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import geometry
from geopandas import GeoDataFrame
import geopandas as gpd
import folium
from shapely.geometry import Polygon, Point, LinearRing
import matplotlib.pyplot as plt
import os
import overpy
import requests
import json
from OSMPythonTools.nominatim import Nominatim
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass
from INFO_COUNTRIES import countries

In [4]:
from math import radians, cos, sin, asin, sqrt

def haversine(point1, point2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    lon1, lat1 = point1.coords.xy[0][0], point1.coords.xy[1][0]
    lon2, lat2 = point2.coords.xy[0][0], point2.coords.xy[1][0]
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371000
    return c * r

def sum_point(point, dpoint):
    lon1, lat1 = point.coords.xy[0][0], point.coords.xy[1][0]
    return geometry.Point(lon1 + dpoint[0], lat1 + dpoint[1])

def grad_desc(point, polygon, step, dp=1e-6): # градиентный спуск
    point1 = sum_point(point, [dp, 0])
    point2 = sum_point(point, [0, dp])
    dist = distance(point, polygon) 
    dist1 = distance(point1, polygon) # мы наперед не знаем какая именно точка ближайшая, это по сути случайная составляющая 
    dist2 = distance(point2, polygon)
        
    grad = np.array([dist1 - dist, dist2 - dist]) / dp
    point = sum_point(point, -grad * step)
    return point

def distance(point, polygon): # дистанция до полигона - наша целевая функция. 
    pol_ext = LinearRing(polygon.exterior.coords)
    d = pol_ext.project(point)
    p = pol_ext.interpolate(d)
    point_closest = geometry.Point(list(p.coords)[0][0], list(p.coords)[0][1])
    return haversine(point_closest, point)

def search(point, polygon, step_init_1 = 0.1):
    points = []
    if polygon.contains(point):
        return [point]
    while not polygon.contains(point):
         point = grad_desc(point, polygon, step_init_1)
         points.append(point)
         
    return points

In [5]:
data = gpd.GeoDataFrame.from_file("/content/drive/My Drive/made/geo/HW2/data/TM_WORLD_BORDERS-0.3.shp", crs="EPSG:4326")
afghan = data[data.NAME == "Afghanistan"].geometry.values[0] # полигон Афганистана
# put your code here
data.head()

,FIPS,ISO2,ISO3,UN,NAME,AREA,POP2005,REGION,SUBREGION,LON,LAT,geometry
0,AC,AG,ATG,28,Antigua and Barbuda,44,83039,19,29,-61.783,17.078,"MULTIPOLYGON (((-61.68667 17.02444, -61.73806 ..."
1,AG,DZ,DZA,12,Algeria,238174,32854159,2,15,2.632,28.163,"POLYGON ((2.96361 36.80222, 2.98139 36.80694, ..."
2,AJ,AZ,AZE,31,Azerbaijan,8260,8352021,142,145,47.395,40.430,"MULTIPOLYGON (((45.08332 39.76804, 45.26639 39..."
3,AL,AL,ALB,8,Albania,2740,3153731,150,39,20.068,41.143,"POLYGON ((19.43621 41.02107, 19.45055 41.06000..."
4,AM,AM,ARM,51,Armenia,2820,3017661,142,145,44.563,40.534,"MULTIPOLYGON (((45.57305 40.63249, 45.52888 40..."


In [6]:
point = geometry.Point(0, 51.5) # Лондон - отправная точка
points =  search(point, afghan, step_init_1=0.0001)
points = [point] + points

In [7]:
m = folium.Map([0, 51.5], zoom_start=2)
i = 0
for point in points:
    coords = [point.coords.xy[1][0], point.coords.xy[0][0]]
    folium.Marker(location=coords, popup=f'{i} step',).add_to(m)
    i += 1
m

Визуализируйте пошагово предложенный алгоритм при помощи ``Folium``

## Задание №2. Качество жизни (20 баллов).

Для измерения показателя качества жизни в точке, найденной в предыдущем задании, вам необходимо рассчитать следующую сумму расстояний (в метрах):

1. Расстояние от точки до 5 ближайших __*__ банкоматов, находящихся в стране с наибольшим количеством объектов жилой недвижимости
2. Расстояние от точки до 5 ближайших школ, находящихся в стране с наибольшим количеством аптек в столице
3. Расстояние от точки до 5 ближайших кинотеатров, наодящихся в стране с самым большим отношением числа железнодорожных станций к автобусным остановкам в южной части __**__

__*__ При поиске _N_ ближайших объектов обязательно использовать ``R-tree``

__**__ Южной частью страны является территория, находящаяся к югу от множества точек, равноудаленных от самой северной и самой южной точек страны

In [8]:
#!pip install OSMPythonTools
#!pip install overpy
#!pip install Rtree==0.8.3

In [9]:
def get_iso_country(point, data):
    for row in data.values:
        if row[-1].contains(point):
            return row[1]
    return None
isos = list(set([get_iso_country(point, data) for point in points]))
isos # Все страны наших точек. Крым деликатно обошли стороной)

['DE', 'PL', 'RO', 'IR', None, 'TR', 'GB', 'AF']

## Банкоматы

In [10]:
def get_atm(iso): # банкоматы страны
    if iso is None:
        return []
    api = overpy.Overpass()
    r = api.query(f"""
    area["ISO3166-1"="{iso}"][admin_level=2];
    (node["amenity"="atm"](area);
    );
    out center;
    """)
    coords  = [geometry.Point(float(node.lon), float(node.lat)) 
              for node in r.nodes]
    return coords

def get_len_resid(iso): # количество жилых домов
    if iso is None:
        return 0
    overpass_url = "https://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json];
    area["ISO3166-1"="{iso}"][admin_level=2];
    (node["building"="residential"](area);
     node["building"="apartments"](area);
     node["building"="terrace"](area);
     node["building"="house"](area);
     node["building"="detached"](area);
     node["building"="semidetached_house"](area);
    );
    out center;
    """
    response = requests.get(overpass_url, 
                        params={'data': overpass_query})
    data = response.json()
    return len(data["elements"])

R_Earth = 6371000 # m
def geo2vec(point):
    lon, lat = point.coords.xy[0][0], point.coords.xy[1][0]
    lat = np.pi/180 * lat
    lon = np.pi/180 * lon
    return np.array((np.cos(lat) * np.cos(lon), np.cos(lat) * np.sin(lon), np.sin(lat))) * R_Earth
def vec2geo(vec):
    lat = np.arcsin(vec[2] / R_Earth) * 180 / np.pi
    lon = np.arctan2(vec[1], vec[0]) * 180 / np.pi
    return geometry.Point(lon, lat)
  
points_vec = [geo2vec(point) for point in points]

In [16]:
count_build = {}
for iso in isos:
    get_build = 0
    while get_build == 0:
        try:
            count_build[iso] = get_len_resid(iso)
            get_build = 1
            print(iso, ": ", count_build[iso])
        except:
            #print("fail loading...")
            pass ## Результат конечно странный, но вроде сделано все правильно

IR :  30
RO :  32
PL :  534
DE :  1115
GB :  1658
AF :  2511
None :  0
TR :  23


In [20]:
iso_max_count_buil = list(count_build.keys())[np.argmax(list(count_build.values()))]
iso_max_count_buil # в Афганистане больше всего домов. Что впрочем вполне может быть.

'AF'

In [22]:
coord_atm = get_atm(iso_max_count_buil) # все банкоматы Афганистана (вряд ли их много)
coord_atm_vec = [geo2vec(point) for point in coord_atm]

In [24]:
from rtree import index

p = index.Property()
p.dimension = 3
p.dat_extension = 'data'
p.idx_extension = 'index'
idx3d = index.Index("3d_index1", properties=p)

for i, pp in enumerate(coord_atm_vec):
    idx3d.insert(i, tuple(pp), tuple(pp))

sum_distance = []
for i in range(len(points_vec)):
    vec = tuple(points_vec[i])
    closest_atm = list(idx3d.nearest(vec, 5, objects='raw'))[:5] # мы переводим угловые координаты в гринвич, потому что нежелательно считать расстояние сразу по углам
    geo_closest_atms = [vec2geo(ca) for ca in closest_atm]
    sum_ = np.sum([haversine(points[i], geo_closest_atm) for geo_closest_atm in geo_closest_atms])
    sum_distance.append(sum_)
    print(f"Сумма расстояний для точки {i}: {sum_} метров")
    # Сумма уменьшается, что внушает некоторую надежду на верный результат

Сумма расстояний для точки 0: 26182596.445174035 метров
Сумма расстояний для точки 1: 23797793.75722707 метров
Сумма расстояний для точки 2: 21416816.025717933 метров
Сумма расстояний для точки 3: 18870433.121645913 метров
Сумма расстояний для точки 4: 16046282.57648502 метров
Сумма расстояний для точки 5: 12883120.517833391 метров
Сумма расстояний для точки 6: 9363798.435744993 метров
Сумма расстояний для точки 7: 5519146.627980847 метров
Сумма расстояний для точки 8: 1536954.414912919 метров
Сумма расстояний для точки 9: 964429.063894253 метров


## Школы

In [19]:
# put your code here
#pharmacy
def get_pharmacy_capitals(iso, countries=countries): # число аптек в столице
    if iso is None:
        return 0
    for c in countries:
        if c["code"] == iso:
            country = c["name"]
            capital = c["capital"]
            break
    nominatim = Nominatim()
    areaId = nominatim.query(f'{capital}, {country}').areaId()
    overpass = Overpass()
    query = overpassQueryBuilder(area=areaId, elementType='node', selector='"amenity"="pharmacy"', out='count')
    result = overpass.query(query)
    return result.countElements()

def get_school(iso): # школы страны
    if iso is None:
        return []
    api = overpy.Overpass()
    r = api.query(f"""
    area["ISO3166-1"="{iso}"][admin_level=2];
    (node["amenity"="school"](area);
    );
    out center;
    """)
    coords  = [geometry.Point(float(node.lon), float(node.lat)) 
              for node in r.nodes]
    return coords

In [32]:
count_farma = {}
for iso in isos:
    get_farma = 0
    while get_farma == 0:
        try:
            count_farma[iso] = get_pharmacy_capitals(iso)
            get_farma = 1
            print(iso, ": ", count_farma[iso])
        except:
            pass

[nominatim] downloading data: search
[overpass] downloading data: [timeout:25][out:json];area(3606663864)->.searchArea;(node["amenity"="pharmacy"](area.searchArea);); out count;
[nominatim] downloading data: search


IR :  670


[overpass] downloading data: [timeout:25][out:json];area(3600377733)->.searchArea;(node["amenity"="pharmacy"](area.searchArea);); out count;
[nominatim] downloading data: search


RO :  748


[overpass] downloading data: [timeout:25][out:json];area(3602907540)->.searchArea;(node["amenity"="pharmacy"](area.searchArea);); out count;
[nominatim] downloading data: search


...

GB :  789


[overpass] downloading data: [timeout:25][out:json];area(3601676476)->.searchArea;(node["amenity"="pharmacy"](area.searchArea);); out count;
[nominatim] downloading data: search


AF :  0
None :  0


[overpass] downloading data: [timeout:25][out:json];area(3600223422)->.searchArea;(node["amenity"="pharmacy"](area.searchArea);); out count;


TR :  1072


In [33]:
iso_max_count_farma = list(count_farma.keys())[np.argmax(list(count_farma.values()))]
iso_max_count_farma # в Анкаре больше всего аптек. Что впрочем тоже может быть.

'TR'

In [21]:
coord_sch = get_school(iso_max_count_farma) # все школы Турции
coord_sch_vec = [geo2vec(point) for point in coord_sch]

In [22]:
from rtree import index
r = index.Property()
r.dimension = 3
r.dat_extension = 'data'
r.idx_extension = 'index'
idx3d1 = index.Index("3d_index4", properties=r)

for i, pp in enumerate(coord_sch_vec):
    idx3d1.insert(i, tuple(pp), tuple(pp))

sum_distance = []
for i in range(len(points_vec)):
    vec = tuple(points_vec[i])
    closest_sch = list(idx3d1.nearest(vec, 5, objects='raw'))[:5] # мы переводим угловые координаты в гринвич, потому что нежелательно считать расстояние сразу по углам
    geo_closest_schs = [vec2geo(ca) for ca in closest_sch]
    sum_ = np.sum([haversine(points[i], geo_closest_sch) for geo_closest_sch in geo_closest_schs])
    sum_distance.append(sum_)
    print(f"Сумма расстояний для точки {i}: {sum_} метров")
    # Сумма минимальна в Турции, что внушает некоторую надежду на верный результат

Сумма расстояний для точки 0: 11562479.256755073 метров
Сумма расстояний для точки 1: 9483741.188099636 метров
Сумма расстояний для точки 2: 7331702.542485569 метров
Сумма расстояний для точки 3: 5041163.781927573 метров
Сумма расстояний для точки 4: 3013265.2593941237 метров
Сумма расстояний для точки 5: 1218091.992056032 метров
Сумма расстояний для точки 6: 326572.9909823502 метров
Сумма расстояний для точки 7: 2844579.6279742382 метров
Сумма расстояний для точки 8: 7038658.494163554 метров
Сумма расстояний для точки 9: 11198126.466712039 метров


## Кинотеатры


In [12]:
def in_south(points, iso, data):
    polygon = data[data.ISO2 == iso].geometry.values[0]
    if polygon.geom_type == "MultiPolygon":
        lons = []
        lats = []
        for poly in polygon:
            (lon, lat) = poly.exterior.coords.xy
            lons += list(lon)
            lats += list(lat)
    else:
        (lons, lats) = polygon.exterior.coords.xy
    southest_idx = np.argmin(lats)
    northest_idx = np.argmax(lats)
    southest_point = geometry.Point(lons[southest_idx], lats[southest_idx])
    northest_point = geometry.Point(lons[northest_idx], lats[northest_idx])
    return [haversine(point, southest_point) < haversine(point, northest_point) for point in points]


def get_rail_by_bus_stop(iso, data): # отношение станций к остановкам
    if iso is None:
        return 0
    api1 = overpy.Overpass()
    r1 = api1.query(f"""
        area["ISO3166-1"="{iso}"][admin_level=2];
        (node["railway"="station"](area);
        );
        out center;
        """)
    api2 = overpy.Overpass()
    r2 = api2.query(f"""
        area["ISO3166-1"="{iso}"][admin_level=2];
        (node["highway"="bus_stop"](area);
        );
        out center;
        """)
    
    coords_rail  = [geometry.Point(float(node.lon), float(node.lat)) 
              for node in r1.nodes]
    coords_bus  = [geometry.Point(float(node.lon), float(node.lat)) 
              for node in r2.nodes]
    
    coords_rail_in_south = in_south(coords_rail, iso, data)#[in_south(point, iso, data) for point in coords_rail]
    coords_bus_in_south = in_south(coords_bus, iso, data)#[in_south(point, iso, data) for point in coords_bus]
    if sum(coords_bus_in_south) == 0:
        print("NO BUS STOP!")
        return 0
    return sum(coords_rail_in_south) / sum(coords_bus_in_south), coords_rail, coords_bus

def get_cinema(iso): # кинотеатры страны
    if iso is None:
        return []
    api = overpy.Overpass()
    r = api.query(f"""
    area["ISO3166-1"="{iso}"][admin_level=2];
    (node["amenity"="cinema"](area);
    );
    out center;
    """)
    coords  = [geometry.Point(float(node.lon), float(node.lat)) 
              for node in r.nodes]
    return coords

In [41]:
count_rail_bus = {}
for iso in isos:
    get_rb = 0
    while get_rb == 0:
        try:
            count_rail_bus[iso], r, b = get_rail_by_bus_stop(iso, data)
            get_rb = 1
            print(iso, ": ", count_rail_bus[iso])
        except:
            pass # В Германи нет остановок. OSM творит чудеса

IR :  0.09818731117824774
PL :  0.013478923864502777
RO :  0.1608554160855416
NO BUS STOP!
DE :  0
TR :  0.0652645273200347
GB :  0.01383633866465819
AF :  0.0
None :  0


In [43]:
iso_max_rail_bus = list(count_rail_bus.keys())[np.argmax(list(count_rail_bus.values()))]
iso_max_rail_bus # На юге Румынии лучший жд транспорт. (несомненно)

'RO'

In [13]:
coord_cin = get_cinema(iso_max_rail_bus)

In [15]:
coord_cin_vec = [geo2vec(point) for point in coord_cin]

In [17]:
from rtree import index
r = index.Property()
r.dimension = 3
r.dat_extension = 'data'
r.idx_extension = 'index'
idx3d2 = index.Index("3d_index5", properties=r)

for i, pp in enumerate(coord_cin_vec):
    idx3d2.insert(i, tuple(pp), tuple(pp))

sum_distance = []
for i in range(len(points_vec)):
    vec = tuple(points_vec[i])
    closest_cin = list(idx3d2.nearest(vec, 5, objects='raw'))[:5] # мы переводим угловые координаты в гринвич, потому что нежелательно считать расстояние сразу по углам
    geo_closest_cins = [vec2geo(ca) for ca in closest_cin]
    sum_ = np.sum([haversine(points[i], geo_closest_cin) for geo_closest_cin in geo_closest_cins])
    sum_distance.append(sum_)
    print(f"Сумма расстояний для точки {i}: {sum_} метров")
    # Сумма минимальна в Румынии

Сумма расстояний для точки 0: 8312831.47980668 метров
Сумма расстояний для точки 1: 6106313.102443936 метров
Сумма расстояний для точки 2: 3809013.065147459 метров
Сумма расстояний для точки 3: 1347486.6037239286 метров
Сумма расстояний для точки 4: 294459.20788892737 метров
Сумма расстояний для точки 5: 2509816.3412244576 метров
Сумма расстояний для точки 6: 5992851.722171451 метров
Сумма расстояний для точки 7: 9971584.788254403 метров
Сумма расстояний для точки 8: 14189240.698258802 метров
Сумма расстояний для точки 9: 18082954.592941996 метров


## Задание №3. Поездка по Нью-Йорку (маршрут - 20 баллов, визуализация - 10 баллов).

Добраться __на автомобиле__ от входа в ``Central Park`` __Нью-Йорка__ (со стороны ``5th Avenue``) до пересечения ``Water Street`` и ``Washington Street`` в Бруклине (откуда получаются лучшие фото Манхэттенского моста) довольно непросто - разумеется, из-за вечных пробок. Однако еще сложнее это сделать, проезжая мимо школ, где дети то и дело переходят дорогу в неположенном месте.

Вам необходимо построить описанный выше маршрут, избегая на своем пути школы. Визуализируйте данный маршрут (также добавив школы и недоступные для проезда участки дорог) при помощи ``Folium``

Данные о расположении школ Нью-Йорка можно найти [здесь](https://catalog.data.gov/dataset/2019-2020-school-point-locations)

In [ ]:
# put your code here
# Жаль, что не успел сделать, интересное задание...